# This code reads the grid which has risk level and suitable_f and includes all other datasets to the grid and then creates the exact grid. 
## The data I am using here: 
1. Infrastrucutre / Detailed Infrastrucutre
2. Erosion
3. InSAR or Excess Ice+
4. Drained Lakes+
5. Ice wedges+
6. Contaminated sites+

In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import rasterio
from rasterstats import zonal_stats
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats
import os
from shapely.geometry import box


In [3]:
target_crs = 'EPSG:3338'

In [4]:
grid_50 = gpd.read_file('/home/emine2/R121/Wainwright/grids/grids_50m.shp')
# Reproject 
if grid_50.crs != target_crs:
    print(f"Reprojecting grid_50 from {grid_50.crs} to {target_crs}.")
    grid_50 = grid_50.to_crs(target_crs)
grid_50.head()

Reprojecting grid_50 from EPSG:4326 to EPSG:3338.


,risk_level,suitable_f,drained_la,excess_ice,ice_wedge_,contaminat,building,road,geometry
0,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301149.894, -230947.77 2..."
1,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301199.894, -230947.77 2..."
2,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301249.894, -230947.77 2..."
3,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301299.894, -230947.77 2..."
4,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301349.894, -230947.77 2..."


## 1. Infrastructure

In [5]:
infrastructure = gpd.read_file('/home/emine2/R121/Wainwright/infrastructure/detailed_infrastructure/w_infrastructure/w_detailed_infrastructure.shp')
if infrastructure.crs != target_crs:
    print(f"Reprojecting infrastructure from {infrastructure.crs} to {target_crs}.")
    infrastructure = infrastructure.to_crs(target_crs)
infrastructure.head()

Reprojecting infrastructure from EPSG:6393 to EPSG:3338.


,OBJECTID,Community,Infr_Type,Source,SHAPE_Leng,Infr_Group,geometry
0,21892.0,Wainwright,Centerline,DCRA,141.673006,Transportation,"LINESTRING (-227838.135 2304987.372, -227739.7..."
1,21893.0,Wainwright,Centerline,DCRA,163.122196,Transportation,"LINESTRING (-227643.014 2304942.695, -227756.2..."
2,21894.0,Wainwright,Centerline,DCRA,499.010263,Transportation,"LINESTRING (-228047.42 2304801.45, -227674.355..."
3,21895.0,Wainwright,Centerline,DCRA,23.698657,Transportation,"LINESTRING (-227613.418 2305159.294, -227629.8..."
4,21896.0,Wainwright,Centerline,DCRA,112.433303,Transportation,"LINESTRING (-227550.97 2305211.291, -227472.93..."


In [6]:
grid_50['infra_exist'] = 0
grid_50['infra_type'] = None

# Perform spatial join to find intersections
intersections = gpd.sjoin(grid_50, infrastructure, predicate='intersects')

# Get unique grid indices that intersect
intersecting_grids = intersections.index.unique()

# Set infra_exist to 1 for intersecting grids
grid_50.loc[intersecting_grids, 'infra_exist'] = 1

# Group by grid index and get unique infra types as comma-separated string
types_per_grid = intersections.groupby(level=0)['Infr_Type'].unique().apply(lambda x: ', '.join(sorted(x)))

# Set infra_type for intersecting grids
grid_50.loc[types_per_grid.index, 'infra_type'] = types_per_grid

# Optionally, view the updated head
grid_50.head()

   risk_level  suitable_f  drained_la  excess_ice  ice_wedge_ contaminat  \
0         0.0        True         NaN         0.0           0       None   
1         0.0        True         NaN         0.0           0       None   
2         0.0        True         NaN         0.0           0       None   
3         0.0        True         NaN         0.0           0       None   
4         0.0        True         NaN         0.0           0       None   

   building  road                                           geometry  \
0         0     0  POLYGON ((-230947.77 2301149.894, -230947.77 2...   
1         0     0  POLYGON ((-230947.77 2301199.894, -230947.77 2...   
2         0     0  POLYGON ((-230947.77 2301249.894, -230947.77 2...   
3         0     0  POLYGON ((-230947.77 2301299.894, -230947.77 2...   
4         0     0  POLYGON ((-230947.77 2301349.894, -230947.77 2...   

   infra_exist infra_type  
0            0       None  
1            0       None  
2            0       None 

## 2. Erosion Forecast

In [7]:
erosion_forecast = gpd.read_file('/home/emine2/R121/Wainwright/erosion/W_erosion_forecast/W_erosion_forecast.shp')
if erosion_forecast.crs != target_crs:
    print(f"Reprojecting erosion_forecast from {erosion_forecast.crs} to {target_crs}.")
    erosion_forecast = erosion_forecast.to_crs(target_crs)
erosion_forecast.head()

Reprojecting erosion_forecast from EPSG:6393 to EPSG:3338.


,OBJECTID,Community,YearStart,YearEnd,SHAPE_Leng,SHAPE_Area,geometry
0,118.0,Wainwright,2019,2039,17303.977273,45640.651803,"MULTIPOLYGON (((-230112.195 2303099.186, -2301..."
1,119.0,Wainwright,2039,2059,17302.613705,44954.637131,"MULTIPOLYGON (((-230175.342 2303050.278, -2301..."
2,120.0,Wainwright,2059,2079,17337.070417,44979.079859,"MULTIPOLYGON (((-230174.878 2303049.777, -2301..."


In [8]:
# Initialize new columns
grid_50['erosion_exist'] = 0

# Perform spatial join to find intersections
intersections = gpd.sjoin(grid_50, erosion_forecast, predicate='intersects')

# Get unique grid indices that intersect
intersecting_grids = intersections.index.unique()

# Set erosion_exist to 1 for intersecting grids
grid_50.loc[intersecting_grids, 'erosion_exist'] = 1

# Group by grid index and get unique YearStart values as comma-separated string
years_per_grid = intersections.groupby(level=0)['YearStart'].unique().apply(lambda x: ', '.join(map(str, sorted(x))))

# Set erosion_year for intersecting grids
grid_50.loc[years_per_grid.index, 'erosion_year'] = years_per_grid

# Optionally, view the updated head
grid_50.head()

,risk_level,suitable_f,drained_la,excess_ice,ice_wedge_,contaminat,building,road,geometry,infra_exist,infra_type,erosion_exist,erosion_year
0,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301149.894, -230947.77 2...",0,None,0,NaN
1,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301199.894, -230947.77 2...",0,None,0,NaN
2,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301249.894, -230947.77 2...",0,None,0,NaN
3,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301299.894, -230947.77 2...",0,None,0,NaN
4,0.0,True,NaN,0.0,0,None,0,0,"POLYGON ((-230947.77 2301349.894, -230947.77 2...",0,None,0,NaN


# 3. InSAR

In [9]:
raster_path = '/home/emine2/R121/data/sar/Wainwright/e_mean_period.tif'

# Open the original raster
with rasterio.open(raster_path) as src:
    raster_crs = src.crs
    nodata = src.nodata
    data = src.read(1)  # Assume single band
    transform = src.transform
    width = src.width
    height = src.height

# If CRS mismatch, reproject in memory
if raster_crs != target_crs:
    print(f"CRS mismatch: Raster CRS is {raster_crs}, Target CRS is {target_crs}. Reprojecting raster in memory.")
    
    # Calculate new transform and dimensions
    new_transform, new_width, new_height = calculate_default_transform(
        raster_crs, target_crs, width, height, *src.bounds
    )
    
    # Create destination array
    dest_data = np.empty((new_height, new_width), dtype=data.dtype)
    
    # Reproject
    reproject(
        source=data,
        destination=dest_data,
        src_transform=transform,
        src_crs=raster_crs,
        src_nodata=nodata,
        dst_transform=new_transform,
        dst_crs=target_crs,
        dst_nodata=nodata,
        resampling=Resampling.nearest  # Adjust as needed (e.g., bilinear for continuous)
    )
    
    # Use reprojected data and transform
    use_data = dest_data
    use_transform = new_transform
    use_nodata = nodata
else:
    print("Raster CRS matches target.")
    use_data = data
    use_transform = transform
    use_nodata = nodata

# Perform zonal statistics
stats = zonal_stats(
    grid_50,
    use_data,
    affine=use_transform,
    stats=['mean'],
    nodata=use_nodata
)

# Add the mean values to a new column named 'excess_ice', handling None as NaN
grid_50['excess_ice'] = [s['mean'] if s['mean'] is not None else float('nan') for s in stats]

# Optionally, view the updated head
print(grid_50.head())

CRS mismatch: Raster CRS is EPSG:4326, Target CRS is EPSG:3338. Reprojecting raster in memory.


/home/emine2/.local/lib/python3.10/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


   risk_level  suitable_f  drained_la  excess_ice  ice_wedge_ contaminat  \
0         0.0        True         NaN         0.0           0       None   
1         0.0        True         NaN         NaN           0       None   
2         0.0        True         NaN         0.0           0       None   
3         0.0        True         NaN         0.0           0       None   
4         0.0        True         NaN         0.0           0       None   

   building  road                                           geometry  \
0         0     0  POLYGON ((-230947.77 2301149.894, -230947.77 2...   
1         0     0  POLYGON ((-230947.77 2301199.894, -230947.77 2...   
2         0     0  POLYGON ((-230947.77 2301249.894, -230947.77 2...   
3         0     0  POLYGON ((-230947.77 2301299.894, -230947.77 2...   
4         0     0  POLYGON ((-230947.77 2301349.894, -230947.77 2...   

   infra_exist infra_type  erosion_exist erosion_year  
0            0       None              0          NaN 

# 4. Drained Lakes - We already have this --  Not going to rewrite the code. 

# Cut the grid_50 for the extends of e_mena-period whih is the Sar data

In [10]:
grid_50.to_file('/home/emine2/R121/Wainwright/grids/grid50_with_all_data.shp')

/tmp/ipykernel_1407801/1409313993.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_50.to_file('/home/emine2/R121/Wainwright/grids/grid50_with_all_data.shp')
/home/emine2/.local/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'infra_exist' to 'infra_exis'
  ogr_write(
/home/emine2/.local/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'erosion_exist' to 'erosion_ex'
  ogr_write(
/home/emine2/.local/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'erosion_year' to 'erosion_ye'
  ogr_write(
